In [1]:
!pip install keras-nlp -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.5/573.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 851.9/851.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 19.2 MB/s eta 0:00:00


In [2]:
import keras_nlp
import tensorflow as tf
import tensorflow_datasets as tfds
import sklearn.cluster as cluster

from tensorflow import keras

policy = keras.mixed_precision.Policy("mixed_float16")
keras.mixed_precision.set_global_policy(policy)

Using TensorFlow backend


In [3]:
TRAIN_BATCH_SIZE = 6
VALIDATION_BATCH_SIZE = 8

TRAIN_NUM_BATCHS = 300
VALIDATION_NUM_BATCHS = 40

AUTOTUNE = tf.data.experimental.AUTOTUNE


def change_range(x):
    return (x / 2.5) - 1


def prepare_dataset(dataset, num_batchs, batch_size):
    dataset = dataset.map(
        lambda z: (
            [z["sentence1"], z["sentence2"]],
            [tf.cast(change_range(z["label"]), tf.float32)],
        ),
        num_parallel_calls=AUTOTUNE,
    )
    dataset = dataset.batch(batch_size)
    dataset = dataset.take(num_batchs)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset


stsb_ds = tfds.load(
    "glue/stsb",
)
stsb_train, stsb_valid = stsb_ds["train"], stsb_ds["validation"]

stsb_train = prepare_dataset(stsb_train, TRAIN_NUM_BATCHS, TRAIN_BATCH_SIZE)
stsb_valid = prepare_dataset(stsb_valid, VALIDATION_NUM_BATCHS, VALIDATION_BATCH_SIZE)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/5749 [00:00<?, ? examples/s]

...

Generating validation examples...:   0%|          | 0/1500 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/glue/stsb/2.0.0.incompleteOFGBNT/glue-validation.tfrecord*...:   0%|      …

Generating test examples...:   0%|          | 0/1379 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/glue/stsb/2.0.0.incompleteOFGBNT/glue-test.tfrecord*...:   0%|          | …

Dataset glue downloaded and prepared to /root/tensorflow_datasets/glue/stsb/2.0.0. Subsequent calls will reuse this data.


Let's see examples from the dataset of two sentenses and their similarity.

In [4]:
for x, y in stsb_train:
    for i, example in enumerate(x):
        print(f"sentence 1 : {example[0]} ")
        print(f"sentence 2 : {example[1]} ")
        print(f"similarity : {y[i]} \n")
    break

sentence 1 : b"A young girl is sitting on Santa's lap." 
sentence 2 : b"A little girl is sitting on Santa's lap" 
similarity : [0.9200001] 

sentence 1 : b'A women sitting at a table drinking with a basketball picture in the background.' 
sentence 2 : b'A woman in a sari drinks something while sitting at a table.' 
similarity : [0.03999996] 

sentence 1 : b'Norway marks anniversary of massacre' 
sentence 2 : b"Norway Marks Anniversary of Breivik's Massacre" 
similarity : [0.52] 

sentence 1 : b'US drone kills six militants in Pakistan: officials' 
sentence 2 : b'US missiles kill 15 in Pakistan: officials' 
similarity : [-0.03999996] 

sentence 1 : b'On Tuesday, the central bank left interest rates steady, as expected, but also declared that overall risks were weighted toward weakness and warned of deflation risks.' 
sentence 2 : b"The central bank's policy board left rates steady for now, as widely expected, but surprised the market by declaring that overall risks were weighted toward 

In [5]:
preprocessor = keras_nlp.models.RobertaPreprocessor.from_preset("roberta_base_en")
backbone = keras_nlp.models.RobertaBackbone.from_preset("roberta_base_en")
inputs = keras.Input(shape=(1), dtype="string", name="sentence")
x = preprocessor(inputs)
h = backbone(x)
embedding = keras.layers.GlobalAveragePooling1D(name="pooling_layer")(
    h, x["padding_mask"]
)
n_embedding = tf.linalg.normalize(embedding, axis=1)[0]
roberta_normal_encoder = keras.Model(inputs=inputs, outputs=n_embedding)

roberta_normal_encoder.summary()

496436344/496436344 [==============================] - 8s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 sentence (InputLayer)       [(None, 1)]                  0         []                            
                                                                                                  
 roberta_preprocessor (Robe  {'token_ids': (None, 512),   0         ['sentence[0][0]']            
 rtaPreprocessor)             'padding_mask': (None, 51                                           
                             2)}                                                                  
                                                                                                  
 roberta_backbone (RobertaB  (None, None, 768)            1240527   ['roberta_preprocessor[0][0]',
 ackbone)                  

#### Build the Siamese network with the regression objective function.

It's described above that the Siamese network has two or more subnetworks, and for this
Siamese model, we need two encoders. But we don't have two encoders; we have only one
encoder, but we will pass the two sentences through it. That way, we can have two paths
to get the embeddings and also shared weights between the two paths.

After passing the two sentences to the model and getting the normalized embeddings, we
will multiply the two normalized embeddings to get the cosine similarity between the two
sentences.

In [6]:

class RegressionSiamese(keras.Model):
    def __init__(self, encoder, **kwargs):
        inputs = keras.Input(shape=(2), dtype="string", name="sentences")
        sen1, sen2 = tf.split(inputs, num_or_size_splits=2, axis=1, name="split")
        u = encoder(sen1)
        v = encoder(sen2)
        cosine_similarity_scores = tf.matmul(u, tf.transpose(v))

        super().__init__(
            inputs=inputs,
            outputs=cosine_similarity_scores,
            **kwargs,
        )

        self.encoder = encoder

    def get_encoder(self):
        return self.encoder


#### Fit the model

Let's try this example before training and compare it to the output after training.

In [7]:
sentences = [
    "Today is a very sunny day.",
    "I am hungry, I will get my meal.",
    "The dog is eating his food.",
]
query = ["The dog is enjoying his meal."]

encoder = roberta_normal_encoder

sentence_embeddings = encoder(tf.constant(sentences))
query_embedding = encoder(tf.constant(query))

cosine_similarity_scores = tf.matmul(query_embedding, tf.transpose(sentence_embeddings))
for i, sim in enumerate(cosine_similarity_scores[0]):
    print(f"cosine similarity score between sentence {i+1} and the query = {sim} ")

cosine similarity score between sentence 1 and the query = 0.966796875 
cosine similarity score between sentence 2 and the query = 0.9765625 
cosine similarity score between sentence 3 and the query = 0.9931640625 


In [8]:
roberta_regression_siamese = RegressionSiamese(roberta_normal_encoder)

roberta_regression_siamese.compile(
    loss=keras.losses.MeanSquaredError(),
    optimizer=keras.optimizers.Adam(2e-5),
)

roberta_regression_siamese.fit(stsb_train, validation_data=stsb_valid, epochs=1)

300/300 [==============================] - 412s 1s/step - loss: 0.3952 - val_loss: 0.4010


In [40]:
roberta_regression_siamese.save("roberta_siasemes.pb")

In [17]:
encoder = roberta_regression_siamese.get_encoder()

In [38]:
sentences = [
    "It is a truth universally acknowledged, that a single man in possession of a good fortune must be in want of a wife.",
    "Everyone knows that a wealthy, unmarried man must be seeking a wife.",
    "In a dystopian future, a young heroine rises to challenge an oppressive regime.",
    "The old castle atop the hill stood as a silent witness to the passing of centuries.",
    "As the sun dipped below the horizon, the sky was painted in shades of pink and orange.",
    "The aroma of freshly baked bread filled the air, enticing everyone in the neighborhood.",
    "The little girl giggled with delight as she chased after the colorful butterflies.",
    "In the heart of the bustling city, a hidden garden offered a peaceful retreat.",
    "The waves crashed against the shore, creating a soothing melody that calmed the soul.",
    "Lost in thought, the poet sat under the ancient oak tree, penning verses about love and loss.",
    "With each step, the adventurer grew closer to the mysterious treasure hidden in the jungle.",
    "The wise elder shared stories of wisdom and life experiences with the eager young minds.",
    "In the depths of the ocean, an array of marine creatures danced in harmony.",
    "The artist meticulously painted every detail, bringing the canvas to life with vibrant colors.",
    "A gentle breeze rustled the leaves, whispering secrets to the enchanted forest.",
    "The detective carefully examined the crime scene, searching for clues to solve the mystery.",
    "Amidst the chaos of the battlefield, a hero emerged to lead the troops to victory.",
    "The starlit sky dazzled with its brilliance, captivating all who gazed upon it.",
    "The laughter of children echoed through the playground on a sunny afternoon.",
    "As the rain fell softly outside, the cozy fireplace warmed the hearts of the family inside.",
    "In a quaint little village, the villagers gathered to celebrate the annual harvest festival.",
    "The scent of fresh flowers wafted through the air, signaling the arrival of spring.",
    "In a land far away, a young prince embarked on a quest to rescue his beloved princess.",
    "The ancient ruins held the secrets of a long-forgotten civilization waiting to be discovered.",
    "The symphony orchestra played in perfect harmony, enchanting the audience with their music.",
    "The majestic mountains stood tall, their peaks touching the heavens.",
    "In a magical realm, mythical creatures roamed freely, unseen by human eyes.",
    "The first snow of winter painted the world in a blanket of white.",
    "The sweet melody of the piano filled the room, evoking emotions in the listeners.",
    "The scent of freshly brewed coffee welcomed visitors into the cozy cafe.",
    "In the heart of the forest, a hidden waterfall cascaded down the rocks.",
    "The talented chef prepared a culinary masterpiece that delighted the taste buds.",
    "The laughter and chatter of friends filled the air during a joyous reunion.",
    "As the full moon rose, the night came alive with mystical allure.",
    "In a quaint bookstore, book lovers lost themselves in a world of literature.",
    "The gentle touch of a mother's hand reassured her child, calming all fears.",
    "In the open field, wildflowers swayed in the gentle breeze like a colorful dance.",
    "The ancient castle held a treasure trove of history within its walls.",
    "The clock struck midnight, marking the beginning of a new day.",
    "In the crowded market, vendors showcased their wares, attracting eager buyers.",
    "The wise old owl perched on a branch, observing the world with knowing eyes.",
    "The sunflowers stretched towards the sun, basking in its warm embrace.",
    "A shooting star streaked across the night sky, granting wishes to those who saw it.",
    "The circus came to town, bringing joy and wonder to children and adults alike.",
    "The scent of pine filled the air, signaling the arrival of the festive holiday season.",
    "In a quaint village, the annual fair brought laughter and merriment to all.",
    "The gentle waves lapped against the boat as it sailed peacefully on the calm sea.",
    "The artisan crafted intricate designs, turning ordinary objects into works of art.",
    "In the meadow, a field of wild horses galloped freely, a sight of untamed beauty.",
    "The young astronaut gazed in awe at the vastness of space from the spaceship window.",
    "The vibrant market was a tapestry of colors, scents, and sounds from around the world.",
    "The old lighthouse stood tall, guiding ships safely through the treacherous waters.",
    "In the heart of the city, street performers entertained passersby with their talents.",
    "The elderly couple held hands, their love enduring through the passage of time.",
    "It is a truth universally acknowledged, that a single man in possession of a good fortune must be in want of a wife.",
    "Everyone knows that a wealthy, unmarried man must be seeking a wife.",
    "In a dystopian future, a young heroine rises to challenge an oppressive regime,"
    "In a galaxy far away, a battle between the forces of good and evil raged on.",
    "The delicate petals of the cherry blossoms fluttered in the spring breeze.",
    "As the train departed, loved ones waved goodbye from the platform.",
    "In the enchanting forest, fairies danced under the glow of the moon.",
    "The brave knight faced the fearsome dragon in an epic showdown.",
    "The aroma of freshly brewed tea filled the cozy teahouse.",
    "The children built sandcastles on the beach, their laughter echoing in the air.",
    "Amidst the city's hustle and bustle, a street musician played a soulful melody.",
    "The gentle rain pitter-pattered on the windowpane, creating a soothing rhythm.",
    "In the heart of the museum, priceless artifacts showcased the past's wonders.",
    "The wise guru imparted wisdom to his disciples on the mountaintop.",
    "The aroma of spices filled the air as the chef prepared an exotic feast.",
    "In the vast savannah, a herd of elephants roamed freely under the golden sun.",
    "The full moon illuminated the night sky, casting a magical glow.",
    "The young boy marveled at the intricate details of his favorite toy.",
    "As the storm subsided, a rainbow arched across the sky, a symbol of hope.",
    "The bustling marketplace was a treasure trove of goods from distant lands.",
    "In the tranquil garden, meditation brought peace to the troubled mind.",
    "The graceful ballerina twirled and leaped, captivating the audience.",
    "The scent of pine and cinnamon filled the air during the holiday season.",
    "The wise owl shared its knowledge with the curious woodland creatures.",
    "In the artist's studio, creativity flowed onto the canvas like a river.",
    "The magnificent waterfall cascaded down the cliff, creating a breathtaking sight.",
    "The storyteller enchanted the crowd with tales of mythical creatures and adventures.",
    "The old library housed dusty tomes of ancient knowledge waiting to be explored.",
    "In the cozy cottage, the fireplace crackled, providing warmth on a chilly night.",
    "The melody of the violin filled the concert hall, evoking emotions in the audience.",
    "The farmer tended to the fields, sowing seeds of hope for a bountiful harvest.",
    "In a distant realm, a sorcerer wielded magic to protect the kingdom from darkness.",
    "The scent of rain-soaked earth filled the air after a refreshing summer shower.",
    "The adventurer set sail on a grand voyage, seeking new lands and discoveries.",
    "In the heart of the desert, an oasis provided refuge to weary travelers.",
    "The ancient temple stood as a testament to the devotion of generations past.",
    "The autumn leaves painted the landscape in hues of red, orange, and gold.",
    "In the quiet village, the sound of church bells signaled the start of a new day.",
    "The circus performers dazzled the audience with their acrobatic feats and tricks.",
    "The scent of freshly baked apple pie wafted from the kitchen, tempting all.",
    "In the meadow, butterflies flitted among the wildflowers in a mesmerizing dance.",
    "The historian carefully preserved ancient artifacts, safeguarding history for the future.",
    "The sound of laughter filled the schoolyard as children played during recess.",
    "In the dimly lit cave, stalactites and stalagmites formed over millennia.",
    "The aroma of a home-cooked meal welcomed family members to the dinner table.",
    "The charming village was straight out of a storybook, with cobblestone streets and quaint cottages.",
    "In a distant land, a hero embarked on a perilous journey to save their kingdom.",
    "The gentle waves lapped at the shore, creating patterns in the sand.",
    "The ancient scroll revealed the hidden knowledge of an ancient civilization.",
    "In the vibrant city, street artists adorned walls with colorful murals.",
    "The scent of fresh lavender filled the air, soothing the senses.",
    "The brave firefighter rushed into the burning building to rescue those trapped inside.",
    "In the dense jungle, exotic creatures roamed freely, hidden from human eyes.",
    "The delicate music box played a hauntingly beautiful melody, evoking nostalgia.",
]

sentence_embeddings = encoder(tf.constant(sentences))

In [39]:
import time
start_time = time.time()

query = ["Everyone knows that a wealthy, unmarried man must be seeking a wife."]
query_embedding = encoder(tf.constant(query))

cosine_simalarities = tf.matmul(query_embedding, tf.transpose(sentence_embeddings))
for i, sim in enumerate(cosine_simalarities[0]):
    print(f"cosine similarity between sentence {i+1} and the query = {sim} ")

end_time = time.time()
execution_time = end_time - start_time
print("Execution Time:", execution_time, "seconds")

cosine similarity between sentence 1 and the query = 0.445556640625 
cosine similarity between sentence 2 and the query = 1.0 
cosine similarity between sentence 3 and the query = 0.165771484375 
cosine similarity between sentence 4 and the query = 0.14306640625 
cosine similarity between sentence 5 and the query = 0.064697265625 
cosine similarity between sentence 6 and the query = 0.16162109375 
cosine similarity between sentence 7 and the query = 0.0199127197265625 
cosine similarity between sentence 8 and the query = 0.177001953125 
cosine similarity between sentence 9 and the query = 0.0946044921875 
cosine similarity between sentence 10 and the query = 0.186279296875 
cosine similarity between sentence 11 and the query = 0.049896240234375 
cosine similarity between sentence 12 and the query = 0.2086181640625 
cosine similarity between sentence 13 and the query = 0.208251953125 
cosine similarity between sentence 14 and the query = 0.12066650390625 
cosine similarity between sente